## Setting up Gemini Chat model 

In [1]:
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# Set up API key
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

# Initialize the model
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")  

Enter your Google API key:  ········


In [5]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hi there! How can I help you today? \n'

### Setting up Tavily Search tool

In [3]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)

tools = [search]

 ········


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1730909496, 'localtime': '2024-11-06 08:11'}, 'current': {'last_updated_epoch': 1730908800, 'last_updated': '2024-11-06 08:00', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 14.1, 'wind_kph': 22.7, 'wind_degree': 22, 'wind_dir': 'NNE', 'pressure_mb': 1015.0, 'pressure_in': 29.98, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 30, 'cloud': 25, 'feelslike_c': 12.1, 'feelslike_f': 53.8, 'windchill_c': 11.5, 'windchill_f': 52.6, 'heatindex_c': 13.1, 'heatindex_f': 55.6, 'dewpoint_c': -0.3, 'dewpoint_f': 31.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.3, 'gust_mph': 23.6, 'gust_kph': 37.9}}"}, {'url': 'https://www.timeanddate.com/weath

In [6]:
model_with_tools = model.bind_tools(tools)

In [10]:
from langchain_core.messages import HumanMessage

response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': '403693a0-9ec4-4f93-a462-456a7a37b690', 'type': 'tool_call'}]


In [12]:
print(f"ContentString: {response}")

ContentString: content='' additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in SF"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-2f67a981-4402-4246-ad3e-200291f4a8a4-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': '403693a0-9ec4-4f93-a462-456a7a37b690', 'type': 'tool_call'}] usage_metadata={'input_tokens': 82, 'output_tokens': 22, 'total_tokens': 104, 'input_token_details': {'cache_read': 0}}


### Creating agent

In [5]:

import langgraph
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [9]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='c8a83af3-a005-455b-9d35-1100351b95c4'),
 AIMessage(content='Hello! 👋 How can I help you today? 😊 \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-4f3aa051-5e2d-4d0d-95d9-19c87d57b4e3-0', usage_metadata={'input_tokens': 76, 'output_tokens': 11, 'total_tokens': 87, 'input_token_details': {'cache_read': 0}})]

In [11]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='d9dc8004-3b2e-4e51-ad27-9d5dc3a63f12'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in sf"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-a2860388-be57-4587-9f01-9d08c15d86e2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'c861a374-93d6-4b15-9d69-3eaa140b004b', 'type': 'tool_call'}], usage_metadata={

### Adding in memory

In [14]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

Config is an essential part of the model, helps track the conversation and provides state to the agent 

In [16]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! 👋  How can I help you today? 😊 \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-8b400cc0-fc7f-454a-b8fc-21fc54022287-0', usage_metadata={'input_tokens': 78, 'output_tokens': 13, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}})]}}
----


In [23]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You said your name is Bob! 😊 \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-782adcb3-2c6f-4a42-b1e4-d16d4d997d59-0', usage_metadata={'input_tokens': 99, 'output_tokens': 8, 'total_tokens': 107, 'input_token_details': {'cache_read': 0}})]}}
----


#### Memory access

In [37]:
memory.get(config=config)

{'v': 1,
 'ts': '2024-11-06T16:26:42.817145+00:00',
 'id': '1ef9c5be-87bf-6cc3-8004-6f3ed9763edc',
 'channel_values': {'messages': [HumanMessage(content='hi im bob!', additional_kwargs={}, response_metadata={}, id='5d0d0828-b3b9-491c-80ee-c60182f75163'),
   AIMessage(content='Hello Bob! 👋  How can I help you today? 😊 \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-8b400cc0-fc7f-454a-b8fc-21fc54022287-0', usage_metadata={'input_tokens': 78, 'output_tokens': 13, 'total_tokens': 91, 'input_token_details': {'cache